# Silver Layer Transformation - Session Results

## Objective
Transform the `bronze.session_results` table into a clean Silver table with correct data types and standardized columns.

## Steps
1. **Read** data from Bronze table `bronze.session_results`.
2. **Select & Rename** key columns:
   - Driver info: `DriverId`, `DriverNumber`, `FirstName`, `LastName`, `FullName`, `Abbreviation`, `BroadcastName`, `CountryCode`
   - Team info: `TeamId`, `TeamName`, `TeamColor`
   - Session info: `session_key`, `session_name`, `event_name`, `year`, `round`
   - Performance: `GridPosition`, `Position`, `ClassifiedPosition`, `Points`, `Laps`, `Time`, `Status`
   - Qualifying: `Q1`, `Q2`, `Q3`
3. **Data Cleaning**
   - Cast numeric fields (`year`, `round`, `DriverNumber`, `GridPosition`, `Position`, `ClassifiedPosition`, `Points`, `Laps`) to integers or floats as needed.
   - Convert time fields (`Time`, `Q1`, `Q2`, `Q3`) into string or numeric durations.
   - Standardize text columns by trimming spaces.
   - Drop duplicates.
4. **Write** results into Silver schema as `silver.session_results`.

## Output
- Clean Silver table:
  `silver.session_results`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
session_results_bronze = spark.table("bronze.session_results")

# Transformations
session_results_silver = (
    session_results_bronze
    .select(
        F.col("DriverId").alias("driver_id"),
        F.col("DriverNumber").cast("int").alias("driver_number"),
        F.trim(F.col("FirstName")).alias("first_name"),
        F.trim(F.col("LastName")).alias("last_name"),
        F.trim(F.col("FullName")).alias("full_name"),
        F.trim(F.col("BroadcastName")).alias("broadcast_name"),
        F.trim(F.col("Abbreviation")).alias("abbreviation"),
        F.trim(F.col("CountryCode")).alias("country_code"),

        F.col("TeamId").alias("team_id"),
        F.trim(F.col("TeamName")).alias("team_name"),
        F.trim(F.col("TeamColor")).alias("team_color"),

        F.col("session_key").cast("int"),
        F.trim(F.col("session_name")).alias("session_name"),
        F.trim(F.col("event_name")).alias("event_name"),
        F.col("year").cast("int").alias("year"),
        F.col("round").cast("int").alias("round"),

        F.col("GridPosition").cast("int").alias("grid_position"),
        F.col("Position").cast("int").alias("position"),
        F.col("ClassifiedPosition").cast("int").alias("classified_position"),
        F.col("Points").cast("float").alias("points"),
        F.col("Laps").cast("int").alias("laps"),
        F.trim(F.col("Time")).alias("race_time"),
        F.trim(F.col("Status")).alias("status"),

        F.trim(F.col("Q1")).alias("q1_time"),
        F.trim(F.col("Q2")).alias("q2_time"),
        F.trim(F.col("Q3")).alias("q3_time"),

        F.trim(F.col("HeadshotUrl")).alias("headshot_url")
    )
    .dropDuplicates()
)

# Write to Silver
(
    session_results_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.session_results")
)